## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [3]:
lines = list(open('news/news_train.txt', 'r', encoding='utf-8'))

In [4]:
import random
random.shuffle(lines)

In [5]:
with open('news/news_test.txt', 'w', encoding='utf-8') as f:
    for line in lines[15000:18000]:
        f.write(line)

In [6]:
from collections import Counter

Counter([line.split('\t')[0] for line in lines[:15000]])

Counter({'culture': 2053,
         'science': 2156,
         'sport': 2215,
         'media': 2111,
         'economics': 2080,
         'life': 2033,
         'style': 284,
         'forces': 1225,
         'business': 554,
         'travel': 289})

In [7]:
'https://drive.google.com/file/d/1mG3tPS_59pANrgwd6T2IgnHWgph4vYbg/view?usp=sharing'

'https://drive.google.com/file/d/1mG3tPS_59pANrgwd6T2IgnHWgph4vYbg/view?usp=sharing'

# 1

In [35]:
import re

import numpy as np        
import pymorphy2
from tqdm.notebook import tqdm

In [36]:
def get_data(path):
    with open(path, "r") as f:
        for line in f:
            label, headline, text = line.strip().split("\t")
            yield label, headline, text

In [37]:
def tokenize_text(text):
    text = text.lower()
    words = re.findall(r'\b\w+\b', text.lower())
    return words

In [38]:
def prepare_data(path):
    data = []
    for label, headline, text in get_data(path):
        item = {}
        item["label"] = label
        item["headline"] = tokenize_text(headline)
        item["text"] = [tokenize_text(sentance) for sentance in re.split(r"[.!?]", text) if len(sentance) > 10]
        data.append(item)
    return data

In [39]:
data_train = prepare_data("news/news_train.txt")
data_test = prepare_data("news/news_test.txt")
data_train[0]

{'label': 'sport',
 'headline': ['овечкин',
  'пожертвовал',
  'детской',
  'хоккейной',
  'школе',
  'автомобиль'],
 'text': [['нападающий',
   'вашингтон',
   'кэпиталз',
   'александр',
   'овечкин',
   'передал',
   'детской',
   'хоккейной',
   'школе',
   'автомобиль',
   'полученный',
   'им',
   'после',
   'окончания',
   'матча',
   'всех',
   'звезд',
   'национальной',
   'хоккейной',
   'лиги',
   'нхл'],
  ['об', 'этом', 'сообщается', 'на', 'официальном', 'сайте', 'лиги'],
  ['автомобиль',
   'honda',
   'accord',
   'был',
   'подарен',
   'хоккеисту',
   'по',
   'решению',
   'спонсоров',
   'мероприятия'],
  ['игрок',
   'нхл',
   'пожертвовал',
   'машину',
   'спортивной',
   'школе',
   'nova',
   'cool',
   'cats',
   'special',
   'hockey',
   'inc'],
  ['которая', 'расположена', 'в', 'штате', 'вирджиния'],
  ['овечкин',
   'общается',
   'с',
   '10',
   'летней',
   'девочкой',
   'анной',
   'шоб',
   'с',
   'синдромом',
   'дауна',
   'которая',
   'занимает

In [40]:
def postprocess_tokens_(data):
    morph = pymorphy2.MorphAnalyzer()
    for item in tqdm(data):
        item["headline"] = [morph.parse(word)[0].normal_form for word in item["headline"]] 
        item["text"] = [[morph.parse(word)[0].normal_form for word in sentance] for sentance in item["text"]] 

postprocess_tokens_(data_train)
postprocess_tokens_(data_test)

data_train[0]

{'label': 'sport',
 'headline': ['овечкин',
  'пожертвовать',
  'детский',
  'хоккейный',
  'школа',
  'автомобиль'],
 'text': [['нападать',
   'вашингтон',
   'кэпиталзти',
   'александр',
   'овечкин',
   'передать',
   'детский',
   'хоккейный',
   'школа',
   'автомобиль',
   'получить',
   'они',
   'после',
   'окончание',
   'матч',
   'весь',
   'звезда',
   'национальный',
   'хоккейный',
   'лига',
   'нхл'],
  ['о', 'это', 'сообщаться', 'на', 'официальный', 'сайт', 'лига'],
  ['автомобиль',
   'honda',
   'accord',
   'быть',
   'подарить',
   'хоккеист',
   'по',
   'решение',
   'спонсор',
   'мероприятие'],
  ['игрок',
   'нхл',
   'пожертвовать',
   'машина',
   'спортивный',
   'школа',
   'nova',
   'cool',
   'cats',
   'special',
   'hockey',
   'inc'],
  ['который', 'расположить', 'в', 'штат', 'вирджиния'],
  ['овечкин',
   'общаться',
   'с',
   '10',
   'летний',
   'девочка',
   'анна',
   'чтоб',
   'с',
   'синдром',
   'даун',
   'который',
   'заниматься',
  

# 2

In [41]:
from gensim.models import Word2Vec

sentences = [item["headline"] for item in data_train]
sentences.extend([sentance for item in data_train for sentance in item["text"]])

w2v = Word2Vec(sentences, workers=8)

In [42]:
print(w2v.wv.most_similar(positive=["день"]))

[('неделя', 0.7838059067726135), ('месяц', 0.7492319345474243), ('десятилетие', 0.6621922254562378), ('час', 0.6401009559631348), ('сразу', 0.6271474361419678), ('полгода', 0.5845887064933777), ('сутки', 0.5722845792770386), ('выходной', 0.5639757513999939), ('раз', 0.5376249551773071), ('утро', 0.5272594094276428)]


In [43]:
print(w2v.wv.most_similar(positive=["спорт"]))

[('мутко', 0.6716254949569702), ('виталий', 0.6587919592857361), ('экспресс', 0.6397702693939209), ('самбо', 0.6362255811691284), ('озеров', 0.6245059967041016), ('зинэтула', 0.6195200681686401), ('комиссар', 0.6120494604110718), ('думский', 0.6111888885498047), ('еврокомиссар', 0.6024603247642517), ('туризм', 0.5985016822814941)]


In [44]:
print(w2v.wv.most_similar(positive=["песня"]))

[('композиция', 0.9322769045829773), ('клип', 0.8946064710617065), ('трек', 0.8596563339233398), ('альбом', 0.8572019338607788), ('сборник', 0.8183584809303284), ('обложка', 0.8161231875419617), ('сингл', 0.8154301643371582), ('комикс', 0.8136640191078186), ('рассказ', 0.7994940280914307), ('пластинка', 0.7938862442970276)]


# 3

In [45]:
max_item_len = 140
label2idx = {}

def word2vec(word):
    if word in w2v.wv:
        return w2v.wv[word]
    return np.zeros((w2v.wv.vector_size,))

def prepare_data(data):
    X = []
    y = []

    for item in tqdm(data):
        label = item["label"]
        headline = item["headline"]
        text = item["text"]

        label_idx = label2idx.get(label, len(label2idx))
        label2idx[label] = label_idx

        word_idx = 0
        sent_idx = 0
        pos_in_sent = 0

        x = np.zeros(140*100)

        while word_idx < max_item_len:
            if word_idx < len(headline):
                x[100 * word_idx: 100 * word_idx + 100] = word2vec(headline[word_idx])
                word_idx += 1
            else:
                if pos_in_sent < len(text[sent_idx]):
                    x[100 * word_idx: 100 * word_idx + 100] = word2vec(text[sent_idx][pos_in_sent])
                    word_idx += 1
                    pos_in_sent += 1
                elif sent_idx < len(text) - 1:
                    sent_idx += 1
                    pos_in_sent = 0
                else:
                    word_idx += 1

        X.append(x)
        y.append(label_idx)
    
    return np.array(X), np.array(y)

X_train, y_train = prepare_data(data_train)
X_test, y_test = prepare_data(data_test)

In [46]:
X_train.shape, y_train.shape, y_train.min(), y_train.max()

((15000, 14000), (15000,), 0, 9)

In [47]:
X_test.shape, y_test.shape, y_test.min(), y_test.max()

((3000, 14000), (3000,), 0, 9)

In [48]:
def prepere_tfidf(data):
    X = []
    y = []

    for item in tqdm(data):
        label = item["label"]
        headline = item["headline"]
        text = item["text"]

        label_idx = label2idx.get(label, len(label2idx))
        label2idx[label] = label_idx

        word_idx = 0
        sent_idx = 0
        pos_in_sent = 0

        x = []

        while word_idx < max_item_len:
            if word_idx < len(headline):
                x.append(headline[word_idx])
                word_idx += 1
            else:
                if pos_in_sent < len(text[sent_idx]):
                    x.append(text[sent_idx][pos_in_sent])
                    word_idx += 1
                    pos_in_sent += 1
                elif sent_idx < len(text) - 1:
                    sent_idx += 1
                    pos_in_sent = 0
                else:
                    x.append("PLACEHOLDER")
                    word_idx += 1

        X.append(" ".join(x))
        y.append(label_idx)
    
    return X, y

X_train_idf, y_train = prepere_tfidf(data_train)
X_test_idf, y_test = prepere_tfidf(data_test)

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_train_idf = tfidf.fit_transform(X_train_idf)
X_test_idf = tfidf.transform(X_test_idf)

In [50]:
X_train_idf.shape

(15000, 75061)

In [51]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(n_jobs=-1)
clf.fit(X_train_idf, y_train)
preds = clf.predict(X_test_idf)
print(f"accuracy = {(y_test == preds).mean()}")

accuracy = 0.859


In [52]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train_idf, y_train)
preds = clf.predict(X_test_idf)
print(f"accuracy = {(y_test == preds).mean()}")

accuracy = 0.798
